In [9]:
!pip install lazypredict
!pip install openpyxl

/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [1]:
import lazypredict
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [2]:
df = pd.read_csv("datasets/edited/davidson_with_classes.csv")
del df['positive']
del df['negative']
del df['neutral']
df

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


## FASTEXT

In [3]:
data = pd.read_csv("datasets/edited/fasttext_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [4]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [5]:
y = dataframe['target']
X = dataframe
del X['target']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:36<00:00,  3.34s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.30
DecisionTreeClassifier,0.99,0.88,None,0.99,1.98
LinearSVC,0.99,0.88,None,0.99,2.33
LogisticRegression,0.99,0.86,None,0.99,1.08
BaggingClassifier,0.99,0.85,None,0.99,11.97
Perceptron,0.98,0.80,None,0.98,0.47
CalibratedClassifierCV,0.99,0.80,None,0.99,10.34
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.46
NearestCentroid,0.78,0.79,None,0.80,0.17


In [7]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [02:55<00:00,  6.07s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [03:01<00:00,  6.25s/it]


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:53<00:00,  5.99s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:52<00:00,  5.95s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.64,0.36,0.26,0.43,0.36,0.64,0.30,0.52
1,1,BaggingClassifier,0.99,0.91,0.97,0.99,0.91,0.99,0.94,0.99
2,1,BernoulliNB,0.87,0.78,0.78,0.88,0.78,0.87,0.76,0.87
3,1,CalibratedClassifierCV,0.99,0.85,0.87,0.99,0.85,0.99,0.86,0.99
4,1,DecisionTreeClassifier,0.99,0.91,0.96,0.99,0.91,0.99,0.93,0.99
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
246,10,RidgeClassifierCV,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
247,10,SGDClassifier,0.97,0.69,0.72,0.97,0.69,0.97,0.71,0.97
248,10,SVC,0.97,0.80,0.84,0.97,0.80,0.97,0.82,0.97


In [10]:
result.to_csv("results/10-Fold-fasttext-davidson.csv")
result.to_excel("results/10-Fold-fasttext-davidson.xlsx")

## GLOVE

In [11]:
data = pd.read_csv("datasets/edited/glove_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.11,0.31,0.35,-0.49,-0.07,0.55,-0.28,0.41,-0.01,0.26,...,-0.04,-0.28,0.10,-0.17,-0.13,0.49,-0.17,0.07,0.41,0.52
1,0.09,-0.02,0.22,-0.41,-0.36,0.09,-0.06,0.10,0.06,-0.08,...,0.25,0.12,-0.22,-0.19,-0.05,0.19,-0.33,0.03,0.43,0.54
2,0.04,0.08,0.33,-0.47,-0.28,0.48,-0.20,-0.01,0.03,-0.19,...,0.19,0.13,-0.27,-0.11,0.01,0.13,-0.03,0.00,0.03,0.97
3,-0.08,0.32,0.11,-0.39,-0.00,0.05,0.07,0.11,-0.70,-0.00,...,0.03,0.06,0.03,-0.10,-0.28,0.17,-0.08,0.06,0.73,0.21
4,0.00,0.06,0.47,-0.53,-0.34,0.24,-0.06,-0.07,0.26,-0.06,...,-0.09,0.03,-0.31,-0.37,-0.24,0.09,-0.02,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.40,0.20,-0.38,0.02,0.43,0.08,0.04,-0.09,0.06,...,0.05,-0.21,-0.09,-0.31,-0.25,0.20,0.38,0.01,0.12,0.87
24779,0.08,0.12,0.50,-0.44,-0.31,0.41,-0.06,0.27,0.14,0.02,...,-0.04,-0.45,-0.27,-0.01,-0.03,0.22,0.08,0.01,0.10,0.89
24780,-0.08,0.28,0.22,-0.41,-0.41,0.17,-0.02,0.02,0.32,-0.20,...,-0.17,-0.10,-0.39,-0.12,-0.22,0.34,0.10,0.04,0.39,0.57
24781,0.04,0.13,0.80,-0.28,-0.26,0.26,0.08,0.06,0.10,-0.46,...,-0.12,-0.01,-0.20,-0.41,-0.02,0.05,0.12,0.01,0.11,0.88


In [12]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,-0.11,0.31,0.35,-0.49,-0.07,0.55,-0.28,0.41,-0.01,0.26,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.09,-0.02,0.22,-0.41,-0.36,0.09,-0.06,0.10,0.06,-0.08,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.04,0.08,0.33,-0.47,-0.28,0.48,-0.20,-0.01,0.03,-0.19,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,-0.08,0.32,0.11,-0.39,-0.00,0.05,0.07,0.11,-0.70,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.06,0.47,-0.53,-0.34,0.24,-0.06,-0.07,0.26,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.40,0.20,-0.38,0.02,0.43,0.08,0.04,-0.09,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.08,0.12,0.50,-0.44,-0.31,0.41,-0.06,0.27,0.14,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.08,0.28,0.22,-0.41,-0.41,0.17,-0.02,0.02,0.32,-0.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.04,0.13,0.80,-0.28,-0.26,0.26,0.08,0.06,0.10,-0.46,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [13]:
y = dataframe['target']
X = dataframe
del X['target']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:33<00:00,  3.23s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.25
DecisionTreeClassifier,0.99,0.90,None,0.99,1.96
BaggingClassifier,0.99,0.85,None,0.99,11.64
LinearSVC,0.99,0.81,None,0.99,2.22
CalibratedClassifierCV,0.99,0.81,None,0.99,11.15
LogisticRegression,0.99,0.80,None,0.99,0.99
PassiveAggressiveClassifier,0.98,0.77,None,0.98,0.50
Perceptron,0.98,0.76,None,0.98,0.50
GaussianNB,0.83,0.76,None,0.86,0.19


In [14]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [02:52<00:00,  5.94s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [02:51<00:00,  5.92s/it]


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:46<00:00,  5.74s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:46<00:00,  5.73s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.64,0.36,0.28,0.45,0.36,0.64,0.30,0.52
1,1,BaggingClassifier,0.99,0.91,0.97,0.99,0.91,0.99,0.94,0.99
2,1,BernoulliNB,0.77,0.66,0.52,0.82,0.66,0.77,0.56,0.79
3,1,CalibratedClassifierCV,0.99,0.89,0.92,0.99,0.89,0.99,0.90,0.99
4,1,DecisionTreeClassifier,0.99,0.93,0.92,0.99,0.93,0.99,0.92,0.99
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.94,0.59,0.59,0.92,0.59,0.94,0.59,0.93
246,10,RidgeClassifierCV,0.94,0.59,0.59,0.92,0.59,0.94,0.59,0.93
247,10,SGDClassifier,0.97,0.70,0.73,0.97,0.70,0.97,0.71,0.97
248,10,SVC,0.97,0.79,0.86,0.97,0.79,0.97,0.81,0.97


In [15]:
result.to_csv("results/10-Fold-glove-davidson.csv")
result.to_excel("results/10-Fold-glove-davidson.xlsx")

## BERT

In [16]:
data = pd.read_csv("datasets/edited/BERT_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0,0,0,0,0,0,0,0.07,0.41,0.52
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0,0,0,0,0,0,0,0.03,0.43,0.54
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0,0,0,0,0,0,0,0.00,0.03,0.97
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0,0,0,0,0,0,0,0.06,0.73,0.21
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0,0,0,0,0,0,0,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0,0,0,0,0,0,0,0.01,0.12,0.87
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0,0,0,0,0,0,0,0.01,0.10,0.89
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0,0,0,0,0,0,0,0.04,0.39,0.57
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0,0,0,0,0,0,0,0.01,0.11,0.88


In [17]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [18]:
y = dataframe['target']
X = dataframe
del X['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:35<00:00,  3.30s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.99,None,0.99,0.24
BaggingClassifier,0.99,0.92,None,0.99,1.66
DecisionTreeClassifier,0.99,0.90,None,0.99,0.36
LinearSVC,0.99,0.89,None,0.99,3.37
LogisticRegression,0.99,0.88,None,0.99,0.87
CalibratedClassifierCV,0.99,0.85,None,0.99,46.08
PassiveAggressiveClassifier,0.98,0.85,None,0.98,0.45
Perceptron,0.98,0.84,None,0.98,0.41
NearestCentroid,0.86,0.83,None,0.87,0.19


In [19]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [02:55<00:00,  6.04s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [03:03<00:00,  6.31s/it]


...

Processing Fold 9 ...


100%|██████████| 29/29 [03:00<00:00,  6.24s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [03:00<00:00,  6.22s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.64,0.36,0.26,0.43,0.36,0.64,0.30,0.52
1,1,BaggingClassifier,1.00,0.94,0.99,1.00,0.94,1.00,0.96,1.00
2,1,BernoulliNB,0.83,0.66,0.66,0.86,0.66,0.83,0.64,0.84
3,1,CalibratedClassifierCV,0.99,0.93,0.99,0.99,0.93,0.99,0.96,0.99
4,1,DecisionTreeClassifier,0.99,0.90,0.95,0.99,0.90,0.99,0.90,0.99
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
246,10,RidgeClassifierCV,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
247,10,SGDClassifier,0.98,0.71,0.73,0.97,0.71,0.98,0.72,0.98
248,10,SVC,0.98,0.80,0.86,0.98,0.80,0.98,0.82,0.98


In [20]:
result.to_csv("results/10-Fold-BERT-davidson.csv")
result.to_excel("results/10-Fold-BERT-davidson.xlsx")

## WORD2VEC

In [21]:
data = pd.read_csv("datasets/edited/word2vec_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.04,0.13,0.04,-0.05,-0.02,0.03,-0.06,-0.05,0.10,0.06,...,-0.10,-0.06,0.06,-0.09,0.02,0.04,-0.05,0.07,0.41,0.52
1,-0.01,0.15,0.05,0.05,-0.06,-0.02,-0.05,-0.10,0.02,0.09,...,-0.19,-0.01,0.07,0.06,-0.00,0.16,-0.07,0.03,0.43,0.54
2,-0.02,0.04,0.10,0.16,-0.11,0.05,-0.04,-0.20,0.10,0.07,...,-0.16,-0.10,0.08,-0.00,0.04,0.03,-0.08,0.00,0.03,0.97
3,-0.16,0.12,0.09,0.16,-0.05,0.02,-0.01,-0.17,-0.08,0.03,...,-0.02,-0.02,0.12,-0.02,-0.02,0.14,-0.05,0.06,0.73,0.21
4,0.05,-0.03,0.10,0.11,-0.13,0.11,-0.01,-0.07,0.13,0.09,...,-0.05,-0.12,0.04,0.09,0.12,0.11,-0.16,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.08,-0.04,0.11,0.15,0.03,-0.05,-0.01,-0.04,0.03,0.17,...,-0.12,-0.13,0.03,-0.01,-0.07,0.07,-0.11,0.01,0.12,0.87
24779,-0.01,0.03,-0.02,0.09,-0.09,0.04,0.07,-0.04,0.04,0.13,...,-0.13,-0.09,0.01,0.14,0.03,-0.05,0.07,0.01,0.10,0.89
24780,-0.04,0.04,0.02,0.18,-0.06,0.04,0.01,-0.04,0.03,0.12,...,-0.16,-0.00,-0.05,0.15,0.06,0.05,-0.17,0.04,0.39,0.57
24781,0.01,0.01,0.08,0.06,0.02,-0.09,-0.02,0.00,0.00,0.21,...,-0.06,-0.15,0.00,0.10,0.04,0.13,-0.13,0.01,0.11,0.88


In [22]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.04,0.13,0.04,-0.05,-0.02,0.03,-0.06,-0.05,0.10,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.01,0.15,0.05,0.05,-0.06,-0.02,-0.05,-0.10,0.02,0.09,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.02,0.04,0.10,0.16,-0.11,0.05,-0.04,-0.20,0.10,0.07,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,-0.16,0.12,0.09,0.16,-0.05,0.02,-0.01,-0.17,-0.08,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.05,-0.03,0.10,0.11,-0.13,0.11,-0.01,-0.07,0.13,0.09,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.08,-0.04,0.11,0.15,0.03,-0.05,-0.01,-0.04,0.03,0.17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,-0.01,0.03,-0.02,0.09,-0.09,0.04,0.07,-0.04,0.04,0.13,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.04,0.04,0.02,0.18,-0.06,0.04,0.01,-0.04,0.03,0.12,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.01,0.01,0.08,0.06,0.02,-0.09,-0.02,0.00,0.00,0.21,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [23]:
y = dataframe['target']
X = dataframe
del X['target']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:34<00:00,  3.25s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.26
LGBMClassifier,0.99,0.87,None,0.99,3.86
LinearSVC,0.99,0.86,None,0.99,2.38
DecisionTreeClassifier,0.99,0.86,None,0.99,1.91
BaggingClassifier,0.99,0.84,None,0.99,11.57
LogisticRegression,0.99,0.83,None,0.99,1.18
CalibratedClassifierCV,0.99,0.82,None,0.99,12.50
Perceptron,0.98,0.80,None,0.98,0.47
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.50


In [24]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [02:45<00:00,  5.70s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [02:44<00:00,  5.67s/it]


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:50<00:00,  5.89s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:47<00:00,  5.76s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.64,0.36,0.26,0.43,0.36,0.64,0.30,0.52
1,1,BaggingClassifier,0.99,0.91,0.96,0.99,0.91,0.99,0.93,0.99
2,1,BernoulliNB,0.80,0.67,0.55,0.83,0.67,0.80,0.58,0.81
3,1,CalibratedClassifierCV,0.99,0.88,0.97,0.99,0.88,0.99,0.90,0.99
4,1,DecisionTreeClassifier,0.99,0.87,0.94,0.99,0.87,0.99,0.88,0.99
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.95,0.60,0.60,0.92,0.60,0.95,0.59,0.93
246,10,RidgeClassifierCV,0.95,0.60,0.60,0.92,0.60,0.95,0.59,0.93
247,10,SGDClassifier,0.97,0.71,0.72,0.97,0.71,0.97,0.71,0.97
248,10,SVC,0.97,0.80,0.86,0.97,0.80,0.97,0.82,0.97


In [26]:
result.to_csv("results/10-Fold-word2vec-davidson.csv")
result.to_excel("results/10-Fold-word2vec-davidson.xlsx")

## TFDIF


In [27]:
data = pd.read_csv("datasets/edited/tfdif_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.41,0.52
1,0.00,0.00,0.35,0.00,0.00,0.00,0.00,0.00,0.00,0.41,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.43,0.54
2,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.97
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.73,0.21
4,0.00,0.00,0.00,0.00,0.00,0.00,0.22,0.00,0.00,0.00,...,0.00,0.00,0.59,0.00,0.00,0.00,0.00,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.12,0.87
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.10,0.89
24780,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.39,0.57
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.11,0.88


In [28]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.00,0.00,0.35,0.00,0.00,0.00,0.00,0.00,0.00,0.41,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.00,0.00,0.00,0.00,0.00,0.22,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [29]:
y = dataframe['target']
X = dataframe
del X['target']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:05<00:00,  2.25s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.28
DecisionTreeClassifier,0.99,0.90,None,0.99,0.27
ExtraTreesClassifier,0.99,0.89,None,0.99,1.65
BaggingClassifier,0.99,0.87,None,0.99,0.95
RandomForestClassifier,0.99,0.86,None,0.99,2.26
PassiveAggressiveClassifier,0.98,0.83,None,0.98,0.46
LogisticRegression,0.99,0.82,None,0.99,0.70
LinearSVC,0.99,0.81,None,0.99,2.32
CalibratedClassifierCV,0.99,0.79,None,0.99,13.10


In [30]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


100%|██████████| 29/29 [01:51<00:00,  3.84s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [01:51<00:00,  3.83s/it]


Processing Fold 3 ...


...

Processing Fold 9 ...


100%|██████████| 29/29 [01:49<00:00,  3.79s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [01:48<00:00,  3.74s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.63,0.26,0.37,0.55,0.26,0.63,0.23,0.51
1,1,BaggingClassifier,0.99,0.94,0.99,0.99,0.94,0.99,0.96,0.99
2,1,BernoulliNB,0.88,0.75,0.69,0.91,0.75,0.88,0.70,0.89
3,1,CalibratedClassifierCV,0.99,0.91,0.99,0.99,0.91,0.99,0.94,0.99
4,1,DecisionTreeClassifier,0.99,0.93,0.99,0.99,0.93,0.99,0.95,0.99
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
246,10,RidgeClassifierCV,0.95,0.59,0.60,0.92,0.59,0.95,0.59,0.93
247,10,SGDClassifier,0.98,0.70,0.72,0.97,0.70,0.98,0.71,0.97
248,10,SVC,0.97,0.74,0.84,0.97,0.74,0.97,0.77,0.97


In [31]:
result.to_csv("results/10-Fold-tfdif-davidson.csv")
result.to_excel("results/10-Fold-tfdif-davidson.xlsx")